<a href="https://colab.research.google.com/github/gseetha04/Transfer-learning/blob/main/Elsevier_fewshot_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate seqeval datasets>=1.8.0 torch>=1.3

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# tokenizer
from transformers import AutoTokenizer, DistilBertTokenizerFast
# sequence tagging model + training-related
from transformers import DistilBertForTokenClassification, Trainer, TrainingArguments
import numpy as np
import pandas as pd
import torch
import json
import sys
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_name = "distilbert-base-cased" # "distilbert-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
# convenience function for wordpiece tokenization of a list of tokens
tokenize = lambda ds: tokenizer(ds, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
RANDOM_SEED = 42

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

In [ ]:
# special tokens
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [ ]:
################SCITE as train and FinCausal as test######################################

In [ ]:
data_train = pd.read_excel('/content/train_data.xlsx',sheet_name = "Sheet5")
data_train.head(4)

,sentences,Bio_label
0,"[""A"",""rare"",""and"",""incurable"",""congenital"",""di...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""E"",""O"",""O"",""..."
1,"[""The"",""soft"",""brown"",""and"",""beige"",""tones"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""O"",""O"",""E"",""..."
2,"[""The"",""JFK"",""baggage"",""system"",""malfunction"",...","[""C"",""C"",""C"",""C"",""C"",""O"",""E"",""O"",""O"",""O"",""O"",""..."
3,"[""Cobalt"",""metal"",""fume"",""and"",""dust"",""cause"",...","[""C"",""C"",""C"",""C"",""C"",""O"",""E"",""O"",""O"",""O"",""O"",""O""]"


In [ ]:
# data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
# data_test.head(4)

In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data_train[["sentences", "Bio_label"]].rename(columns={"sentences": "X", "Bio_label": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]


In [ ]:
for i, x in enumerate(y_raw):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw[i][j] = a.replace('CE', 'O')

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
id2tag

{0: 'E', 1: 'O', 2: 'C'}

In [ ]:
# ensure our data is well-formed:
for ds in [X_raw, X_raw]:
  for row in ds:
    #print(row)
    assert all(len(tok) > 0 for tok in row)

In [ ]:
# # ensure our data is well-formed:
# for ds in [X_dev_raw, X_dev_raw]:
#   for row in ds:
#     #print(row)
#     assert all(len(tok) > 0 for tok in row)

In [ ]:
# # ensure our data is well-formed:
# for ds in [X_raw_test, X_raw_test]:
#   for row in ds:
#     #print(row)
#     assert all(len(tok) > 0 for tok in row)

In [ ]:
def encode_tags(tags, encodings, tag2id):
    """
    See https://huggingface.co/transformers/custom_datasets.html#tok-ner

    >If the tokenizer splits a token into multiple sub-tokens, then we will end up with a mismatch between our tokens and our labels.
    >One way to handle this is to only train on the tag labels for the first subtoken of a split token.
    >We can do this in 🤗 Transformers by setting the labels we wish to ignore to -100.
    >In the example above, if the label for @HuggingFace is 3 (indexing `B-corporation`),
    >we would set the labels of ['@', 'hugging', '##face'] to [3, -100, -100].
    """
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())


    return encoded_labels

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
data_test

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."
4,"[In, addition, to, the, impacts, from, the, Me...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",9,"[ , , , , , , , , , , , , , , , ,..."
...,...,...,...,...
260,"[In, addition, ,, corporate, and, other, gener...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, O, ...",526,"[ , , , , , , , , , , , , , , , , ..."
261,"[The, provision, for, credit, losses, increase...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",528,"[ , , , , , , , , , , , , , , , , , ..."
262,"[Depreciation, and, amortization, increased, p...","[E, E, E, E, O, O, O, C, C, C, C, C, C, C, C, ...",531,"[ , , , , , , , , , , , , , , , ..."
263,"[We, recorded, a, loss, of, $, 13.1, million, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",534,"[ , , , , , , , , , , , , , , , ,..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
test_text = np.array(X_raw_test)
test_label = np.array(y_raw_test)

<ipython-input-15-1c9404d1cf69>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_text = np.array(X_raw_test)
<ipython-input-15-1c9404d1cf69>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_label = np.array(y_raw_test)


In [ ]:
X_train_final = np.array(X_raw)
y_train_final = np.array(y_raw)

<ipython-input-16-aff0ef744a5d>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_final = np.array(X_raw)
<ipython-input-16-aff0ef744a5d>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train_final = np.array(y_raw)


In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    """
    Pytorch dataset for easy batching
    """
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
)

In [ ]:
model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(id2tag)

{0: 'O', 1: 'C', 2: 'E'}


In [ ]:
targetname = ['E','O','C']

In [ ]:
import io
from sklearn.model_selection import KFold
import numpy as np

classification_prediction_list =[]
k=4
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
#notebook_login()
for i, (train_index, test_index) in enumerate(kf.split(test_text, test_label)):
  train_texts, dev_texts =test_text[train_index].tolist(),test_text[test_index].tolist()
  train_labels, dev_labels = test_label[train_index].tolist(),test_label[test_index].tolist()

  print(np.shape(train_texts))
  print(np.shape(dev_texts))
  dev_texts_fi = np.array(dev_texts)
  dev_labels_fi = np.array(dev_labels)

  X_train_app = np.append(X_train_final,dev_texts_fi, axis=0)
  y_train_app = np.append(y_train_final,dev_labels_fi, axis=0)

  appended_train_text = pd.DataFrame(X_train_app)
  appended_train_label = pd.DataFrame(y_train_app)

  appended_train = pd.concat([appended_train_text,appended_train_label],axis=1)
  appended_train.columns = ['text','label']

  # load data and rename relevant columns to X (input) and y (output)
  dataset = appended_train[["text", "label"]].rename(columns={"text": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_train, y_raw_train = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

  #print(X_raw)
  X_text_new = pd.DataFrame(X_raw_train,columns=['sentences'])
  y_text_new = pd.DataFrame(y_raw_train,columns=['label'])

  df_train_new = pd.concat([X_text_new,y_text_new],axis=1)

  # load data and rename relevant columns to X (input) and y (output)
  dataset_new = df_train_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

  unique_tags = set(label for labels in y_raw_test for label in labels)
  tag2id = {tag: id for id, tag in enumerate(unique_tags)}
  id2tag = {id: tag for tag, id in tag2id.items()}
  print(id2tag)

  X_train = tokenize(X_raw_test)
  X_dev = tokenize(train_texts)
  X_test= tokenize(train_texts)

  y_train = encode_tags(tags=y_raw_test, encodings=X_train, tag2id=tag2id)
  y_dev = encode_tags(tags=train_labels, encodings=X_dev, tag2id=tag2id)
  y_test = encode_tags(tags=train_labels, encodings=X_test, tag2id=tag2id)

  train_dataset = CustomDataset(X_train, y_train)
  val_dataset = CustomDataset(X_dev, y_dev)
  test_dataset = CustomDataset(X_test,y_test)

  with io.open("trainfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(X_raw_test))

  with io.open("testfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(train_texts))

  training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
  )

  model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))

  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )
  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)

  flat_list = [item for sublist in preds for item in sublist]
  print(np.shape(flat_list))
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
  print(np.shape(flat_list1))


  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']
  print(frame_combine_test)

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  from sklearn.metrics import classification_report
  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape
<ipython-input-21-1ca74975ba08>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-21-1ca74975ba08>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the nd

(198,)
(67,)
{0: 'E', 1: 'O', 2: 'C'}


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.853400,1.314557
2,0.519700,0.634939
3,0.242100,0.395703


(41580,)
(41580,)
       preds  gold_data
0          2       -100
1          0          0
2          0          0
3          0          0
4          0          0
...      ...        ...
41575      2       -100
41576      2       -100
41577      2       -100
41578      0       -100
41579      2       -100

[41580 rows x 2 columns]
       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
41407      2          2
41408      2          2
41409      2          2
41410      2          2
41411      2          1

[9085 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.90      0.78      0.84      3649
           O       0.52      0.68      0.59       481
           C       0.85      0.91      0.88      4955

    accuracy                           0.84      9085
   macro avg       0.76      0.79      0.77      9085
weighted avg       0.85      

...

Epoch,Training Loss,Validation Loss
1,0.868900,1.421061
2,0.495200,0.663101
3,0.255100,0.464925


(34029,)
(34029,)
       preds  gold_data
0          2       -100
1          0          0
2          0          0
3          0          0
4          2          0
...      ...        ...
34024      0       -100
34025      0       -100
34026      0       -100
34027      0       -100
34028      2       -100

[34029 rows x 2 columns]
       preds  gold_data
1          0          0
2          0          0
3          0          0
4          2          0
5          2          0
...      ...        ...
33923      2          2
33924      2          2
33925      2          2
33926      2          2
33927      2          2

[9133 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.93      0.70      0.80      3797
           O       0.51      0.69      0.59       489
           C       0.80      0.93      0.86      4847

    accuracy                           0.82      9133
   macro avg       0.75      0.77      0.75      9133
weighted avg       0.84      

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape
<ipython-input-21-1ca74975ba08>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-21-1ca74975ba08>:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the nd

Epoch,Training Loss,Validation Loss
1,0.880700,1.461192
2,0.488300,0.660218
3,0.205500,0.447750


(41790,)
(41790,)
       preds  gold_data
0          2       -100
1          0          0
2          0          0
3          0          0
4          0          0
...      ...        ...
41785      2       -100
41786      2       -100
41787      2       -100
41788      2       -100
41789      2       -100

[41790 rows x 2 columns]
       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
41645      2          2
41646      2          2
41647      1          2
41648      2          2
41649      0          2

[9333 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.81      0.90      0.85      3899
           O       0.44      0.76      0.55       465
           C       0.94      0.80      0.86      4969

    accuracy                           0.84      9333
   macro avg       0.73      0.82      0.76      9333
weighted avg       0.86      

In [ ]:
df_train_new

,sentences,label
0,"[""A"", ""rare"", ""and"", ""incurable"", ""congenital""...","[""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""O"", ""O"", ""E"", ..."
1,"[""The"", ""soft"", ""brown"", ""and"", ""beige"", ""tone...","[""C"", ""C"", ""C"", ""C"", ""C"", ""C"", ""O"", ""O"", ""O"", ..."
2,"[""The"", ""JFK"", ""baggage"", ""system"", ""malfuncti...","[""C"", ""C"", ""C"", ""C"", ""C"", ""O"", ""E"", ""O"", ""O"", ..."
3,"[""Cobalt"", ""metal"", ""fume"", ""and"", ""dust"", ""ca...","[""C"", ""C"", ""C"", ""C"", ""C"", ""O"", ""E"", ""O"", ""O"", ..."
4,"[""Both"", ""over-the-counter"", ""pain"", ""pills"", ...","[""C"", ""C"", ""C"", ""C"", ""O"", ""C"", ""C"", ""O"", ""E"", ..."
...,...,...
932,"[""Noninterest"", ""expense"", ""increased"", ""$"", ""...","[""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""O"", ""O"", ""O"", ..."
933,"[""The"", ""average"", ""rate"", ""on"", ""short-term"",...","[""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ..."
934,"[""Food"", ""and"", ""beverage"", ""revenues"", ""decre...","[""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""O"", ..."
935,"[""Casino"", ""expenses"", ""decreased"", ""$"", ""1.03...","[""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ""E"", ..."


In [ ]:
df_train_new.to_excel('/content/train.xlsx')

In [ ]:
test = []
for i in train_texts:
  for j in i:
    #print(j)
    test.append(j)

In [ ]:
train = []
for tr in df_train_new['sentences']:
  for t in tr.split(','):
    train.append(t)

In [ ]:
from collections import Counter
#list1=['apple','egg','apple','banana','egg','apple']
counts_test = Counter(test)
print(counts_test)

Counter({',': 460, 'the': 423, 'of': 343, 'in': 323, 'to': 285, '$': 284, 'and': 273, '.': 262, 'million': 229, 'for': 124, '%': 105, 'a': 101, 'due': 100, 'sales': 100, 'The': 85, 'by': 85, 'from': 83, 'as': 82, 'primarily': 77, '2020': 69, 'was': 64, 'net': 62, 'costs': 61, 'compared': 60, 'income': 57, '2019': 56, 'increase': 51, 'or': 49, 'Company': 46, 'increased': 46, '2012': 46, 'lower': 45, 'billion': 42, 'year': 41, '2011': 41, 'December': 40, '31': 38, 'with': 37, 'offset': 37, 'A': 36, 'related': 34, '&': 34, 'were': 33, 'decrease': 33, 'an': 32, 'SG': 32, 'operating': 31, 'driven': 30, 'higher': 30, 'partially': 30, 'prior': 29, 'which': 29, 'cost': 29, 'our': 29, 'decreased': 28, 'loans': 26, 'In': 24, 'growth': 24, 'result': 24, 'COVID-19': 23, 'during': 22, 'decline': 21, '2010': 21, 'expenses': 20, 'quarter': 20, 'including': 20, 'cash': 20, 'expense': 20, 'restructuring': 20, 'currency': 19, 'losses': 18, 'certain': 18, 'Merger': 18, 'loss': 18, 'activities': 18, '2018

In [ ]:
with open('/content/fincausal_test_datacount.txt','w') as f:
  for tag, count in counts_test.items():
    f.write('{}|{}\n'.format(tag, count))

In [ ]:
train

['["A"',
 ' "rare"',
 ' "and"',
 ' "incurable"',
 ' "congenital"',
 ' "disease"',
 ' "which"',
 ' "causes"',
 ' "blindness"',
 ' "has"',
 ' "been"',
 ' "successfully"',
 ' "treated"',
 ' "for"',
 ' "the"',
 ' "first"',
 ' "time"',
 ' "using"',
 ' "gene"',
 ' "therapy"]',
 '["The"',
 ' "soft"',
 ' "brown"',
 ' "and"',
 ' "beige"',
 ' "tones"',
 ' "in"',
 ' "the"',
 ' "decoration"',
 ' "provide"',
 ' "a"',
 ' "tranquil"',
 ' "mood"]',
 '["The"',
 ' "JFK"',
 ' "baggage"',
 ' "system"',
 ' "malfunction"',
 ' "caused"',
 ' "delays"',
 ' "for"',
 ' "some"',
 ' "flights"',
 ' "and"',
 ' "caused"',
 ' "some"',
 ' "passengers"',
 ' "to"',
 ' "simply"',
 ' "travel"',
 ' "without"',
 ' "their"',
 ' "things"]',
 '["Cobalt"',
 ' "metal"',
 ' "fume"',
 ' "and"',
 ' "dust"',
 ' "cause"',
 ' "irritation"',
 ' "of"',
 ' "the"',
 ' "nose"',
 ' "and"',
 ' "throat"]',
 '["Both"',
 ' "over-the-counter"',
 ' "pain"',
 ' "pills"',
 ' "and"',
 ' "prescription"',
 ' "drugs"',
 ' "cause"',
 ' "medication-overus

In [ ]:
from collections import Counter
#list1=['apple','egg','apple','banana','egg','apple']
counts_test = Counter(train)
print(counts_test)

Counter({' "the"': 1537, ' "of"': 746, ' "and"': 556, '"': 506, ' "by"': 451, ' "caused"': 392, ' "in"': 390, ' "a"': 383, '["The"': 341, ' "to"': 331, ' "from"': 259, ' "is"': 198, ' "was"': 157, ' "that"': 150, ' "on"': 144, ' "': 127, ' "for"': 124, ' "are"': 109, ' "as"': 107, ' "an"': 93, ' "with"': 88, ' "has"': 85, ' "been"': 69, ' "$"': 69, ' "."]': 66, ' "cause"': 59, ' "or"': 59, ' "which"': 54, ' "after"': 54, ' "due"': 49, ' "million"': 48, ' "at"': 47, ' "primarily"': 45, ' "one"': 42, ' "were"': 39, ' "damage"': 36, '["In"': 36, ' "causes"': 35, ' "%"': 35, ' "resulted"': 34, '["A"': 33, ' "but"': 33, ' "sales"': 31, ' "it"': 30, ' "increased"': 29, ' "have"': 29, ' "be"': 28, ' "had"': 27, ' "triggered"': 25, ' "their"': 24, ' "than"': 24, ' "his"': 24, ' "billion"': 24, ' "2020"': 24, ' "pain"': 23, ' "more"': 23, ' "my"': 23, ' "this"': 23, ' "through"': 22, ' "people"': 22, ' "death"': 22, ' "into"': 22, ' "most"': 21, ' "produced"': 21, ' "fire"': 21, ' "our"': 21, '

In [ ]:
with open('/content/scite_train_datacount.txt','w') as f:
  for tag, count in counts_test.items():
    f.write('{}|{}\n'.format(tag, count))

In [ ]:
##############SCITE as train and Organizational as test#####################################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
for i, x in enumerate(y_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_raw[i][j] = a.replace('CT', 'O')

In [ ]:
test_text = np.array(X_raw)
test_label = np.array(y_raw)

<ipython-input-26-42a08091acfc>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_text = np.array(X_raw)
<ipython-input-26-42a08091acfc>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_label = np.array(y_raw)


In [ ]:
print(id2tag)

{0: 'E', 1: 'O', 2: 'C'}


In [ ]:
targetname = ['O', 'C', 'E']

In [ ]:
import io
from sklearn.model_selection import KFold
import numpy as np

classification_prediction_list =[]
k=4
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
#notebook_login()
for i, (train_index, test_index) in enumerate(kf.split(test_text, test_label)):
  train_texts, dev_texts =test_text[train_index].tolist(),test_text[test_index].tolist()
  train_labels, dev_labels = test_label[train_index].tolist(),test_label[test_index].tolist()

  dev_texts_fi = np.array(dev_texts)
  dev_labels_fi = np.array(dev_labels)

  X_train_app = np.append(X_train_final,dev_texts_fi, axis=0)
  y_train_app = np.append(y_train_final,dev_labels_fi, axis=0)

  appended_train_text = pd.DataFrame(X_train_app)
  appended_train_label = pd.DataFrame(y_train_app)

  appended_train = pd.concat([appended_train_text,appended_train_label],axis=1)
  appended_train.columns = ['text','label']

  # load data and rename relevant columns to X (input) and y (output)
  dataset = appended_train[["text", "label"]].rename(columns={"text": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_train, y_raw_train = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

  #print(X_raw)
  X_text_new = pd.DataFrame(X_raw_train,columns=['sentences'])
  y_text_new = pd.DataFrame(y_raw_train,columns=['label'])

  df_train_new = pd.concat([X_text_new,y_text_new],axis=1)

  # load data and rename relevant columns to X (input) and y (output)
  dataset_new = df_train_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

  unique_tags = set(label for labels in y_raw_test for label in labels)
  tag2id = {tag: id for id, tag in enumerate(unique_tags)}
  id2tag = {id: tag for tag, id in tag2id.items()}
  print(id2tag)

  X_train = tokenize(X_raw_test)
  X_dev = tokenize(train_texts)
  X_test= tokenize(train_texts)

  y_train = encode_tags(tags=y_raw_test, encodings=X_train, tag2id=tag2id)
  y_dev = encode_tags(tags=train_labels, encodings=X_dev, tag2id=tag2id)
  y_test = encode_tags(tags=train_labels, encodings=X_test, tag2id=tag2id)

  train_dataset = CustomDataset(X_train, y_train)
  val_dataset = CustomDataset(X_dev, y_dev)
  test_dataset = CustomDataset(X_test,y_test)

  with io.open("trainfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(X_raw_test))

  with io.open("testfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(train_texts))

  training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
  )

  model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))


  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )
  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)

  flat_list = [item for sublist in preds for item in sublist]
  print(np.shape(flat_list))
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
  print(np.shape(flat_list1))


  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']
  print(frame_combine_test)

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  from sklearn.metrics import classification_report
  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()


<ipython-input-28-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-28-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'E', 1: 'O', 2: 'C'}


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.838700,0.989615
2,0.571400,0.649407
3,0.443300,0.565545


(222775,)
(222775,)
        preds  gold_data
0           0       -100
1           1          1
2           2          1
3           2          2
4           2       -100
...       ...        ...
222770      2       -100
222771      2       -100
222772      2       -100
222773      2       -100
222774      2       -100

[222775 rows x 2 columns]
        preds  gold_data
1           1          1
2           2          1
3           2          2
5           2          2
6           2          2
...       ...        ...
222665      2          2
222666      2          2
222667      2          2
222668      2          2
222669      2          2

[54640 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.79      0.93      0.86     21408
           O       0.75      0.50      0.60     12110
           C       0.82      0.83      0.83     21122

    accuracy                           0.80     54640
   macro avg       0.79      0.75      0.76     54640
w

...

Epoch,Training Loss,Validation Loss
1,0.842000,1.011770
2,0.513600,0.670530
3,0.330300,0.542693


(368720,)
(368720,)
        preds  gold_data
0           0       -100
1           1          1
2           1          1
3           2          2
4           2       -100
...       ...        ...
368715      2       -100
368716      2       -100
368717      0       -100
368718      0       -100
368719      0       -100

[368720 rows x 2 columns]
        preds  gold_data
1           1          1
2           1          1
3           2          2
5           2          2
6           2          2
...       ...        ...
368523      2          2
368524      2          2
368525      2          2
368526      2          2
368527      2          2

[54869 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.81      0.91      0.86     21171
           O       0.78      0.45      0.57     12414
           C       0.79      0.89      0.84     21284

    accuracy                           0.80     54869
   macro avg       0.79      0.75      0.76     54869
w

<ipython-input-28-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-28-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.861400,1.021450
2,0.515500,0.670863
3,0.355800,0.547484


(368720,)
(368720,)
        preds  gold_data
0           0       -100
1           1          1
2           2          1
3           2          2
4           2       -100
...       ...        ...
368715      2       -100
368716      2       -100
368717      2       -100
368718      2       -100
368719      2       -100

[368720 rows x 2 columns]
        preds  gold_data
1           1          1
2           2          1
3           2          2
5           2          2
6           2          2
...       ...        ...
368514      2          1
368515      2          2
368516      2          2
368517      2          2
368518      2          2

[55020 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.82      0.90      0.86     21477
           O       0.80      0.45      0.58     12426
           C       0.77      0.89      0.83     21117

    accuracy                           0.80     55020
   macro avg       0.80      0.75      0.75     55020
w

In [ ]:
#####################FinCausal as train and SCITE as test##############################################

In [ ]:
data_train = pd.read_json('/content/fincausal-extra-all-train.json', lines=True)
data_train.head(4)

,words,ner,id,postfix
0,"[It, found, that, total, U.S., health, care, s...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0001.00026,"[ , , , , , , , , , , , , , , , ..."
1,"[Transat, loss, more, than, doubles, as, it, w...","[E, E, E, E, E, O, C, C, C, C, C, C, C]",0003.00001,"[ , , , , , , , , , , , ]"
2,"[MONTREAL, -, Transat, AT, Inc., 's, third-qua...","[O, O, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0003.00002,"[ , , , , , , , , , , , , , , ,..."
3,"[The, Montreal-based, company, lost, $, 11.0, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",0003.00003,"[ , , , , , , , , , , , , , , , ,..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_train = data_train[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_train.A.values], [json.dumps(labels_test) for labels_test in dataset_train.B.values]

X_raw, y_raw = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
unique_tags = set(label for labels in y_raw for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
print(id2tag)

{0: 'O', 1: 'C', 2: 'E'}


In [ ]:
X_train_final = np.array(X_raw)
y_train_final = np.array(y_raw)

<ipython-input-9-aff0ef744a5d>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_final = np.array(X_raw)
<ipython-input-9-aff0ef744a5d>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train_final = np.array(y_raw)


In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw_test[i][j] = a.replace('CE', 'O')

In [ ]:
test_text = np.array(X_raw_test)
test_label = np.array(y_raw_test)

<ipython-input-13-1c9404d1cf69>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_text = np.array(X_raw_test)
<ipython-input-13-1c9404d1cf69>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_label = np.array(y_raw_test)


In [ ]:
print(id2tag)

{0: 'O', 1: 'E', 2: 'C'}


In [ ]:
targetname = ['O', 'E', 'C']

In [ ]:
import io
from sklearn.model_selection import KFold
import numpy as np

classification_prediction_list =[]
k=4
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
#notebook_login()
for i, (train_index, test_index) in enumerate(kf.split(test_text, test_label)):
  train_texts, dev_texts =test_text[train_index].tolist(),test_text[test_index].tolist()
  train_labels, dev_labels = test_label[train_index].tolist(),test_label[test_index].tolist()

  dev_texts_fi = np.array(dev_texts)
  dev_labels_fi = np.array(dev_labels)

  X_train_app = np.append(X_train_final,dev_texts_fi, axis=0)
  y_train_app = np.append(y_train_final,dev_labels_fi, axis=0)

  appended_train_text = pd.DataFrame(X_train_app)
  appended_train_label = pd.DataFrame(y_train_app)

  appended_train = pd.concat([appended_train_text,appended_train_label],axis=1)
  appended_train.columns = ['text','label']

  # load data and rename relevant columns to X (input) and y (output)
  dataset = appended_train[["text", "label"]].rename(columns={"text": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_train, y_raw_train = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

  #print(X_raw)
  X_text_new = pd.DataFrame(X_raw_train,columns=['sentences'])
  y_text_new = pd.DataFrame(y_raw_train,columns=['label'])

  df_train_new = pd.concat([X_text_new,y_text_new],axis=1)

  # load data and rename relevant columns to X (input) and y (output)
  dataset_new = df_train_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

  unique_tags = set(label for labels in y_raw_test for label in labels)
  tag2id = {tag: id for id, tag in enumerate(unique_tags)}
  id2tag = {id: tag for tag, id in tag2id.items()}
  print(id2tag)

  X_train = tokenize(X_raw_test)
  X_dev = tokenize(train_texts)
  X_test= tokenize(train_texts)

  y_train = encode_tags(tags=y_raw_test, encodings=X_train, tag2id=tag2id)
  y_dev = encode_tags(tags=train_labels, encodings=X_dev, tag2id=tag2id)
  y_test = encode_tags(tags=train_labels, encodings=X_test, tag2id=tag2id)

  train_dataset = CustomDataset(X_train, y_train)
  val_dataset = CustomDataset(X_dev, y_dev)
  test_dataset = CustomDataset(X_test,y_test)

  with io.open("trainfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(X_raw_test))

  with io.open("testfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(train_texts))

  training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
  )

  model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))


  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )
  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)

  flat_list = [item for sublist in preds for item in sublist]
  print(np.shape(flat_list))
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
  print(np.shape(flat_list1))


  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']
  print(frame_combine_test)

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  from sklearn.metrics import classification_report
  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()


<ipython-input-19-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-19-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'O', 1: 'E', 2: 'C'}


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.365500,1.406861
2,0.154900,1.030584
3,0.042800,0.795710


...

<ipython-input-19-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-19-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'O', 1: 'E', 2: 'C'}


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.397800,1.597110
2,0.152400,1.247018
3,0.043100,1.290007


(10224,)
(10224,)
       preds  gold_data
0          0       -100
1          0          2
2          0          2
3          2       -100
4          0       -100
...      ...        ...
10219      1       -100
10220      1       -100
10221      1       -100
10222      1       -100
10223      1       -100

[10224 rows x 2 columns]
       preds  gold_data
1          0          2
2          0          2
5          0          2
6          0          2
7          0          2
...      ...        ...
10182      2          2
10183      2          2
10184      2          0
10185      2          0
10186      2          0

[2796 rows x 2 columns]
              precision    recall  f1-score   support

           O       0.79      0.57      0.66      2090
           E       0.17      0.30      0.22       348
           C       0.32      0.59      0.42       358

    accuracy                           0.54      2796
   macro avg       0.43      0.49      0.43      2796
weighted avg       0.65      

In [ ]:
################FinCausal as train and Organizational as test #############################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
for i, x in enumerate(y_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_raw[i][j] = a.replace('CT', 'O')

In [ ]:
test_text = np.array(X_raw)
test_label = np.array(y_raw)

<ipython-input-14-42a08091acfc>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_text = np.array(X_raw)
<ipython-input-14-42a08091acfc>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_label = np.array(y_raw)


In [ ]:
print(id2tag)

{0: 'O', 1: 'C', 2: 'E'}


In [ ]:
targetname = ['O', 'C', 'E']

In [ ]:
import io
from sklearn.model_selection import KFold
import numpy as np

classification_prediction_list =[]
k=4
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
#notebook_login()
for i, (train_index, test_index) in enumerate(kf.split(test_text, test_label)):
  train_texts, dev_texts =test_text[train_index].tolist(),test_text[test_index].tolist()
  train_labels, dev_labels = test_label[train_index].tolist(),test_label[test_index].tolist()

  dev_texts_fi = np.array(dev_texts)
  dev_labels_fi = np.array(dev_labels)

  X_train_app = np.append(X_train_final,dev_texts_fi, axis=0)
  y_train_app = np.append(y_train_final,dev_labels_fi, axis=0)

  appended_train_text = pd.DataFrame(X_train_app)
  appended_train_label = pd.DataFrame(y_train_app)

  appended_train = pd.concat([appended_train_text,appended_train_label],axis=1)
  appended_train.columns = ['text','label']

  # load data and rename relevant columns to X (input) and y (output)
  dataset = appended_train[["text", "label"]].rename(columns={"text": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_train, y_raw_train = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

  #print(X_raw)
  X_text_new = pd.DataFrame(X_raw_train,columns=['sentences'])
  y_text_new = pd.DataFrame(y_raw_train,columns=['label'])

  df_train_new = pd.concat([X_text_new,y_text_new],axis=1)

  # load data and rename relevant columns to X (input) and y (output)
  dataset_new = df_train_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

  unique_tags = set(label for labels in y_raw_test for label in labels)
  tag2id = {tag: id for id, tag in enumerate(unique_tags)}
  id2tag = {id: tag for tag, id in tag2id.items()}
  print(id2tag)

  X_train = tokenize(X_raw_test)
  X_dev = tokenize(train_texts)
  X_test= tokenize(train_texts)

  y_train = encode_tags(tags=y_raw_test, encodings=X_train, tag2id=tag2id)
  y_dev = encode_tags(tags=train_labels, encodings=X_dev, tag2id=tag2id)
  y_test = encode_tags(tags=train_labels, encodings=X_test, tag2id=tag2id)

  train_dataset = CustomDataset(X_train, y_train)
  val_dataset = CustomDataset(X_dev, y_dev)
  test_dataset = CustomDataset(X_test,y_test)

  with io.open("trainfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(X_raw_test))

  with io.open("testfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(train_texts))

  training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
  )

  model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))


  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )
  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)

  flat_list = [item for sublist in preds for item in sublist]
  print(np.shape(flat_list))
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
  print(np.shape(flat_list1))


  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']
  print(frame_combine_test)

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  from sklearn.metrics import classification_report
  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()


<ipython-input-20-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-20-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'O', 1: 'C', 2: 'E'}


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.373100,0.686468
2,0.181800,0.565200
3,0.044500,0.562197


...

<ipython-input-20-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-20-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'O', 1: 'C', 2: 'E'}


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.418700,0.722196
2,0.247700,0.529092
3,0.082000,0.558378


(368720,)
(368720,)
        preds  gold_data
0           2       -100
1           0          0
2           1          0
3           1          1
4           1       -100
...       ...        ...
368715      1       -100
368716      1       -100
368717      1       -100
368718      1       -100
368719      1       -100

[368720 rows x 2 columns]
        preds  gold_data
1           0          0
2           1          0
3           1          1
5           1          1
6           1          1
...       ...        ...
368514      1          0
368515      1          1
368516      1          1
368517      1          1
368518      1          1

[55020 rows x 2 columns]
              precision    recall  f1-score   support

           O       0.72      0.55      0.62     12426
           C       0.80      0.86      0.83     21117
           E       0.84      0.89      0.86     21477

    accuracy                           0.80     55020
   macro avg       0.78      0.77      0.77     55020
w

In [ ]:
import io
from sklearn.model_selection import KFold
import numpy as np

classification_prediction_list =[]
k=4
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
#notebook_login()
for i, (train_index, test_index) in enumerate(kf.split(test_text, test_label)):
  train_texts, dev_texts =test_text[train_index].tolist(),test_text[test_index].tolist()
  train_labels, dev_labels = test_label[train_index].tolist(),test_label[test_index].tolist()

  dev_texts_fi = np.array(dev_texts)
  dev_labels_fi = np.array(dev_labels)

  X_train_app = np.append(X_train_final,dev_texts_fi, axis=0)
  y_train_app = np.append(y_train_final,dev_labels_fi, axis=0)

  appended_train_text = pd.DataFrame(X_train_app)
  appended_train_label = pd.DataFrame(y_train_app)

  appended_train = pd.concat([appended_train_text,appended_train_label],axis=1)
  appended_train.columns = ['text','label']

  # load data and rename relevant columns to X (input) and y (output)
  dataset = appended_train[["text", "label"]].rename(columns={"text": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_train, y_raw_train = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

  #print(X_raw)
  X_text_new = pd.DataFrame(X_raw_train,columns=['sentences'])
  y_text_new = pd.DataFrame(y_raw_train,columns=['label'])

  df_train_new = pd.concat([X_text_new,y_text_new],axis=1)

  # load data and rename relevant columns to X (input) and y (output)
  dataset_new = df_train_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

  unique_tags = set(label for labels in y_raw_test for label in labels)
  tag2id = {tag: id for id, tag in enumerate(unique_tags)}
  id2tag = {id: tag for tag, id in tag2id.items()}
  print(id2tag)

  X_train = tokenize(X_raw_test)
  X_dev = tokenize(train_texts)
  X_test= tokenize(train_texts)

  y_train = encode_tags(tags=y_raw_test, encodings=X_train, tag2id=tag2id)
  y_dev = encode_tags(tags=train_labels, encodings=X_dev, tag2id=tag2id)
  y_test = encode_tags(tags=train_labels, encodings=X_test, tag2id=tag2id)

  train_dataset = CustomDataset(X_train, y_train)
  val_dataset = CustomDataset(X_dev, y_dev)
  test_dataset = CustomDataset(X_test,y_test)

  with io.open("trainfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(X_raw_test))

  with io.open("testfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(train_texts))

  training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
  )

  model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))


  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )
  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)

  flat_list = [item for sublist in preds for item in sublist]
  print(np.shape(flat_list))
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
  print(np.shape(flat_list1))


  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']
  print(frame_combine_test)

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  from sklearn.metrics import classification_report
  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()


<ipython-input-26-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-26-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'O', 1: 'E', 2: 'C'}


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.382400,0.657250
2,0.164700,0.559466
3,0.050600,0.589344


(222775,)
(222775,)
        preds  gold_data
0           1       -100
1           0          0
2           2          0
3           2          2
4           2       -100
...       ...        ...
222770      2       -100
222771      2       -100
222772      2       -100
222773      2       -100
222774      2       -100

[222775 rows x 2 columns]
        preds  gold_data
1           0          0
2           2          0
3           2          2
5           2          2
6           2          2
...       ...        ...
222665      2          2
222666      2          2
222667      2          2
222668      2          2
222669      2          2

[54640 rows x 2 columns]
              precision    recall  f1-score   support

           O       0.67      0.57      0.62     12110
           E       0.84      0.89      0.86     21408
           C       0.81      0.83      0.82     21122

    accuracy                           0.80     54640
   macro avg       0.77      0.76      0.77     54640
w

...

<ipython-input-26-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-26-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'O', 1: 'E', 2: 'C'}


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.383500,0.731669
2,0.203700,0.513758
3,0.078700,0.569191


(368720,)
(368720,)
        preds  gold_data
0           1       -100
1           0          0
2           2          0
3           2          2
4           2       -100
...       ...        ...
368715      2       -100
368716      2       -100
368717      2       -100
368718      2       -100
368719      2       -100

[368720 rows x 2 columns]
        preds  gold_data
1           0          0
2           2          0
3           2          2
5           2          2
6           2          2
...       ...        ...
368514      0          0
368515      2          2
368516      2          2
368517      2          2
368518      2          2

[55020 rows x 2 columns]
              precision    recall  f1-score   support

           O       0.70      0.57      0.63     12426
           E       0.85      0.87      0.86     21477
           C       0.80      0.86      0.83     21117

    accuracy                           0.80     55020
   macro avg       0.78      0.77      0.77     55020
w

In [ ]:
#####################################Organizational as train and SCITE as test#################################################

In [ ]:
# mount Google Drive
from google.colab import drive
import os

drive.mount("/content/drive", force_remount=True)

# data location
#parent_dir = "/content/drive/MyDrive"
parent_dir = "/content/drive/MyDrive/"
mk_data_path = lambda filename: os.path.join(parent_dir, filename)
#data_path = f"{parent_dir}/BERT_data_final.xlsx"
data_path = mk_data_path("BERT_data_final1.xlsx")

# once finished, flush changes to files to make them visible (assuming we write to Google Drive)
# drive.flush_and_unmount()

Mounted at /content/drive


In [ ]:
data = pd.read_excel(data_path)
data.head(4)

,Causal-relation text,BIO Code,"{""words"":Causal-relation text,""ner"":BIO Code}",BIO_Code_changed,BIO Code-changed-NER
0,"[""When "",""a "",""policyholder "",""or "",""insured ""...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""When "",""a "",""policyholder "",""or "",""...","[""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""O"",""...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."
1,"[""During "",""2020 "",""in "",""response "",""to "",""th...","[""O"",""O"",""O"",""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""...","{""words"":[""During "",""2020 "",""in "",""response "",...","[""O"",""O"",""O"",""O"",""O"",""C"",""C"",""C"",""C"",""C"",""C"",""...","[""O"",""O"",""O"",""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-..."
2,"[""Prolonged "",""periods "",""of "",""low "",""interes...","[""B-C"",""I-C"",""I-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT...","{""words"":[""Prolonged "",""periods "",""of "",""low ""...","[""C"",""C"",""C"",""C"",""C"",""C"",""O"",""CT"",""E"",""E"",""O"",...","[""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""..."
3,"[""Conversely "",""a "",""rise "",""in "",""interest "",...","[""O"",""O"",""B-C"",""I-C"",""I-C"",""I-C"",""O"",""B-CT"",""O...","{""words"":[""Conversely "",""a "",""rise "",""in "",""in...","[""O"",""O"",""C"",""C"",""C"",""C"",""O"",""CT"",""O"",""O"",""O"",...","[""O"",""O"",""B-ENTITY"",""B-ENTITY"",""B-ENTITY"",""B-E..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset = data[["Causal-relation text", "BIO_Code_changed"]].rename(columns={"Causal-relation text": "X", "BIO_Code_changed": "y"})
# load each row's data as a list of strings
X_raw, y_raw = [json.loads(tokens) for tokens in dataset.X.values], [json.loads(labels) for labels in dataset.y.values]

In [ ]:
for i, x in enumerate(y_raw):
    for j, a in enumerate(x):
        if 'CT' in a:
            y_raw[i][j] = a.replace('CT', 'O')

In [ ]:
X_train_final = np.array(X_raw)
y_train_final = np.array(y_raw)

<ipython-input-9-aff0ef744a5d>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_final = np.array(X_raw)
<ipython-input-9-aff0ef744a5d>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  y_train_final = np.array(y_raw)


In [ ]:
unique_tags = set(label for labels in y_train_final for label in labels)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [ ]:
print(id2tag)

{0: 'E', 1: 'C', 2: 'O'}


In [ ]:
data_test = pd.read_excel('/content/test_data.xlsx',sheet_name = "Sheet3")
data_test.head(4)

,sentences,Bio_label
0,"[""Various"",""hormonal"","","",""bacterial"",""and"",""i...","[""C"",""C"",""C"",""C"",""C"",""C"",""C"",""O"",""E""]"
1,"[""A"",""stereo"",""buss"",""outputs"",""the"",""stereo"",...","[""C"",""C"",""C"",""O"",""E"",""E"",""E"",""E"",""O"",""C"",""C"",""..."
2,"[""The"",""incoming"",""water"",""caused"",""a"",""stain""...","[""C"",""C"",""C"",""O"",""E"",""E"",""O"",""O"",""O"",""O"",""O"",""..."
3,"[""The"",""genreal"",""anesthetic"",""cause"",""unconsc...","[""C"",""C"",""C"",""O"",""E"",""O"",""E"",""O"",""O"",""O"",""O"",""..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["sentences", "Bio_label"]].rename(columns={"sentences": "A", "Bio_label": "B"})
# load each row's data as a list of strings
X_raw_test, y_raw_test = [json.loads(tokens_test) for tokens_test in dataset_test.A.values], [json.loads(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
for i, x in enumerate(y_raw_test):
    for j, a in enumerate(x):
        if 'CE' in a:
            y_raw_test[i][j] = a.replace('CE', 'O')

In [ ]:
test_text = np.array(X_raw_test)
test_label = np.array(y_raw_test)

<ipython-input-15-1c9404d1cf69>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_text = np.array(X_raw_test)
<ipython-input-15-1c9404d1cf69>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_label = np.array(y_raw_test)


In [ ]:
targetname = ['E', 'C', 'O']

In [ ]:
import io
from sklearn.model_selection import KFold
import numpy as np

classification_prediction_list =[]
k=4
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
#notebook_login()
for i, (train_index, test_index) in enumerate(kf.split(test_text, test_label)):
  train_texts, dev_texts =test_text[train_index].tolist(),test_text[test_index].tolist()
  train_labels, dev_labels = test_label[train_index].tolist(),test_label[test_index].tolist()

  dev_texts_fi = np.array(dev_texts)
  dev_labels_fi = np.array(dev_labels)

  X_train_app = np.append(X_train_final,dev_texts_fi, axis=0)
  y_train_app = np.append(y_train_final,dev_labels_fi, axis=0)

  appended_train_text = pd.DataFrame(X_train_app)
  appended_train_label = pd.DataFrame(y_train_app)

  appended_train = pd.concat([appended_train_text,appended_train_label],axis=1)
  appended_train.columns = ['text','label']

  # load data and rename relevant columns to X (input) and y (output)
  dataset = appended_train[["text", "label"]].rename(columns={"text": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_train, y_raw_train = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

  #print(X_raw)
  X_text_new = pd.DataFrame(X_raw_train,columns=['sentences'])
  y_text_new = pd.DataFrame(y_raw_train,columns=['label'])

  df_train_new = pd.concat([X_text_new,y_text_new],axis=1)

  # load data and rename relevant columns to X (input) and y (output)
  dataset_new = df_train_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

  unique_tags = set(label for labels in y_raw_test for label in labels)
  tag2id = {tag: id for id, tag in enumerate(unique_tags)}
  id2tag = {id: tag for tag, id in tag2id.items()}
  print(id2tag)

  X_train = tokenize(X_raw_test)
  X_dev = tokenize(train_texts)
  X_test= tokenize(train_texts)

  y_train = encode_tags(tags=y_raw_test, encodings=X_train, tag2id=tag2id)
  y_dev = encode_tags(tags=train_labels, encodings=X_dev, tag2id=tag2id)
  y_test = encode_tags(tags=train_labels, encodings=X_test, tag2id=tag2id)

  train_dataset = CustomDataset(X_train, y_train)
  val_dataset = CustomDataset(X_dev, y_dev)
  test_dataset = CustomDataset(X_test,y_test)

  with io.open("trainfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(X_raw_test))

  with io.open("testfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(train_texts))

  training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
  )

  model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))


  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )
  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)

  flat_list = [item for sublist in preds for item in sublist]
  print(np.shape(flat_list))
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
  print(np.shape(flat_list1))


  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']
  print(frame_combine_test)

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  from sklearn.metrics import classification_report
  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()


<ipython-input-20-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-20-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'E', 1: 'C', 2: 'O'}


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.763600,1.204814
2,0.460700,1.106177
3,0.399600,0.817293


...

Epoch,Training Loss,Validation Loss
1,0.667000,1.160689
2,0.437400,1.188714
3,0.377200,0.908083


(10153,)
(10153,)
       preds  gold_data
0          2       -100
1          2          1
2          1          1
3          1          1
4          1       -100
...      ...        ...
10148      2       -100
10149      2       -100
10150      2       -100
10151      2       -100
10152      0       -100

[10153 rows x 2 columns]
       preds  gold_data
1          2          1
2          1          1
3          1          1
5          1          2
7          1          0
...      ...        ...
10109      0          2
10110      0          2
10111      0          1
10112      0          1
10113      0          1

[2666 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.24      0.51      0.33       346
           C       0.33      0.52      0.41       330
           O       0.82      0.58      0.68      1990

    accuracy                           0.57      2666
   macro avg       0.47      0.54      0.47      2666
weighted avg       0.69      

<ipython-input-20-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-20-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.643900,1.266117
2,0.554000,1.244776
3,0.454500,0.913248


(10224,)
(10224,)
       preds  gold_data
0          2       -100
1          2          1
2          1          1
3          1       -100
4          1       -100
...      ...        ...
10219      2       -100
10220      2       -100
10221      0       -100
10222      0       -100
10223      0       -100

[10224 rows x 2 columns]
       preds  gold_data
1          2          1
2          1          1
5          1          1
6          1          1
7          1          1
...      ...        ...
10182      2          1
10183      2          1
10184      2          2
10185      2          2
10186      2          2

[2796 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.24      0.50      0.33       348
           C       0.33      0.66      0.44       358
           O       0.83      0.54      0.66      2090

    accuracy                           0.55      2796
   macro avg       0.47      0.57      0.48      2796
weighted avg       0.70      

In [ ]:
#############################Organizational as train and FinCausal as test #################################################

In [ ]:
data_test = pd.read_json('/content/Fincausal-test.json', lines=True)
data_test.head(4)

,words,ner,id,postfix
0,"[During, 2020, ,, the, Company, also, issued, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",3,"[ , , , , , , , , , , , , , , , , ..."
1,"[The, significant, increases, in, earnings, as...","[E, E, E, E, E, E, E, E, O, O, O, O, C, C, C, ...",4,"[ , , , , , , , , , , , , , , , ..."
2,"[The, provision, for, credit, losses, was, $, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",7,"[ , , , , , , , , , , , , , , , , ..."
3,"[Higher, net, charge-offs, also, contributed, ...","[E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, ...",8,"[ , , , , , , , , , , , , , , , ..."


In [ ]:
# load data and rename relevant columns to X (input) and y (output)
dataset_test = data_test[["words", "ner"]].rename(columns={"words": "A", "ner": "B"})
# load each row's data as a list of strings
s1, s2 = [json.dumps(tokens_test) for tokens_test in dataset_test.A.values], [json.dumps(labels_test) for labels_test in dataset_test.B.values]

In [ ]:
X_raw_test, y_raw_test = [json.loads(token_test) for token_test in s1], [json.loads(label_test) for label_test in s2]

In [ ]:
test_text = np.array(X_raw_test)
test_label = np.array(y_raw_test)

<ipython-input-24-1c9404d1cf69>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_text = np.array(X_raw_test)
<ipython-input-24-1c9404d1cf69>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_label = np.array(y_raw_test)


In [ ]:
import io
from sklearn.model_selection import KFold
import numpy as np

classification_prediction_list =[]
k=4
seed = 1
kf = KFold(n_splits=k,random_state=seed, shuffle=True)
#notebook_login()
for i, (train_index, test_index) in enumerate(kf.split(test_text, test_label)):
  train_texts, dev_texts =test_text[train_index].tolist(),test_text[test_index].tolist()
  train_labels, dev_labels = test_label[train_index].tolist(),test_label[test_index].tolist()

  dev_texts_fi = np.array(dev_texts)
  dev_labels_fi = np.array(dev_labels)

  X_train_app = np.append(X_train_final,dev_texts_fi, axis=0)
  y_train_app = np.append(y_train_final,dev_labels_fi, axis=0)

  appended_train_text = pd.DataFrame(X_train_app)
  appended_train_label = pd.DataFrame(y_train_app)

  appended_train = pd.concat([appended_train_text,appended_train_label],axis=1)
  appended_train.columns = ['text','label']

  # load data and rename relevant columns to X (input) and y (output)
  dataset = appended_train[["text", "label"]].rename(columns={"text": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_train, y_raw_train = [json.dumps(tokens) for tokens in dataset.X.values], [json.dumps(labels) for labels in dataset.y.values]

  #print(X_raw)
  X_text_new = pd.DataFrame(X_raw_train,columns=['sentences'])
  y_text_new = pd.DataFrame(y_raw_train,columns=['label'])

  df_train_new = pd.concat([X_text_new,y_text_new],axis=1)

  # load data and rename relevant columns to X (input) and y (output)
  dataset_new = df_train_new[["sentences", "label"]].rename(columns={"sentences": "X", "label": "y"})
  # load each row's data as a list of strings
  X_raw_test, y_raw_test = [json.loads(tokens) for tokens in dataset_new.X.values], [json.loads(labels) for labels in dataset_new.y.values]

  unique_tags = set(label for labels in y_raw_test for label in labels)
  tag2id = {tag: id for id, tag in enumerate(unique_tags)}
  id2tag = {id: tag for tag, id in tag2id.items()}
  print(id2tag)

  X_train = tokenize(X_raw_test)
  X_dev = tokenize(train_texts)
  X_test= tokenize(train_texts)

  y_train = encode_tags(tags=y_raw_test, encodings=X_train, tag2id=tag2id)
  y_dev = encode_tags(tags=train_labels, encodings=X_dev, tag2id=tag2id)
  y_test = encode_tags(tags=train_labels, encodings=X_test, tag2id=tag2id)

  train_dataset = CustomDataset(X_train, y_train)
  val_dataset = CustomDataset(X_dev, y_dev)
  test_dataset = CustomDataset(X_test,y_test)

  with io.open("trainfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(X_raw_test))

  with io.open("testfile_" + str(i) + ".txt", 'w', encoding='utf-8') as f:
       f.write(str(train_texts))

  training_args = TrainingArguments(
    #output_dir=mk_data_path("results"),   # output directory
    output_dir = '/content/DistilBertforTokenClassification',
    num_train_epochs=3,                   # total number of training epochs
    per_device_train_batch_size=16,       # batch size per device during training
    per_device_eval_batch_size=64,        # batch size for evaluation
    warmup_steps=500,                     # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                    # strength of weight decay
    #logging_dir=mk_data_path("logs"),     # directory for storing logs
    logging_dir= '/content/logs',
    logging_steps=10,
    evaluation_strategy = "epoch",
    #report_to = "wandb"
  )

  model = DistilBertForTokenClassification.from_pretrained(model_name, num_labels=len(unique_tags))


  trainer = Trainer(
    model=model,                          # the instantiated 🤗 Transformers model to be trained
    args=training_args,                   # training arguments, defined above
    train_dataset=train_dataset,          # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    #compute_metrics = compute_metrics
  )
  trainer.train()
  pred = trainer.predict(test_dataset)
  preds = pred.predictions.argmax(-1)

  flat_list = [item for sublist in preds for item in sublist]
  print(np.shape(flat_list))
  flat_list1 = [item1 for sublist1 in y_test for item1 in sublist1]
  print(np.shape(flat_list1))


  lis_preds_frame = pd.DataFrame(flat_list)
  lis_test_labels_frame = pd.DataFrame(flat_list1)
  frame_combine_test = pd.concat([lis_preds_frame,lis_test_labels_frame],axis=1)
  frame_combine_test.columns = ['preds','gold_data']
  print(frame_combine_test)

  frame_combine_test_new = frame_combine_test[frame_combine_test.gold_data != -100]
  print(frame_combine_test_new)

  from sklearn.metrics import classification_report
  print(classification_report(y_true=frame_combine_test_new['gold_data'], y_pred=frame_combine_test_new['preds'], zero_division = 1,target_names = list(targetname))) #id2tag.keys()


<ipython-input-25-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-25-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)


{0: 'E', 1: 'C', 2: 'O'}


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.664800,0.569433
2,0.516300,0.384966
3,0.400300,0.453971


(41580,)
(41580,)
       preds  gold_data
0          0       -100
1          2          0
2          0          0
3          0          0
4          0          0
...      ...        ...
41575      1       -100
41576      1       -100
41577      1       -100
41578      0       -100
41579      1       -100

[41580 rows x 2 columns]
       preds  gold_data
1          2          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
41407      1          1
41408      1          1
41409      1          1
41410      1          1
41411      0          2

[9085 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.79      0.89      0.84      3649
           C       0.96      0.77      0.85      4955
           O       0.40      0.82      0.54       481

    accuracy                           0.82      9085
   macro avg       0.72      0.83      0.74      9085
weighted avg       0.86      

...

Epoch,Training Loss,Validation Loss
1,0.618200,0.618346
2,0.447000,0.407168
3,0.410900,0.326756


(34029,)
(34029,)
       preds  gold_data
0          0       -100
1          0          0
2          0          0
3          0          0
4          0          0
...      ...        ...
34024      0       -100
34025      0       -100
34026      0       -100
34027      0       -100
34028      0       -100

[34029 rows x 2 columns]
       preds  gold_data
1          0          0
2          0          0
3          0          0
4          0          0
5          0          0
...      ...        ...
33923      1          1
33924      1          1
33925      1          1
33926      1          1
33927      1          1

[9133 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.90      0.87      0.88      3797
           C       0.90      0.91      0.90      4847
           O       0.63      0.74      0.68       489

    accuracy                           0.88      9133
   macro avg       0.81      0.84      0.82      9133
weighted avg       0.88      

<ipython-input-25-d488c51d088f>:14: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_texts_fi = np.array(dev_texts)
<ipython-input-25-d488c51d088f>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dev_labels_fi = np.array(dev_labels)
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.599400,0.773856
2,0.434500,0.579534
3,0.418100,0.399356


(41790,)
(41790,)
       preds  gold_data
0          1       -100
1          2          0
2          0          0
3          0          0
4          2          0
...      ...        ...
41785      1       -100
41786      1       -100
41787      1       -100
41788      1       -100
41789      1       -100

[41790 rows x 2 columns]
       preds  gold_data
1          2          0
2          0          0
3          0          0
4          2          0
5          0          0
...      ...        ...
41645      1          1
41646      1          1
41647      1          1
41648      1          1
41649      1          1

[9333 rows x 2 columns]
              precision    recall  f1-score   support

           E       0.87      0.86      0.87      3899
           C       0.89      0.88      0.89      4969
           O       0.48      0.59      0.53       465

    accuracy                           0.86      9333
   macro avg       0.75      0.78      0.76      9333
weighted avg       0.86      